## Merge a HED tag spreadsheet with an existing JSON sidecar.

To use this notebook, substitute the specifics of your BIDS
dataset for the following variables:

| Variable | Purpose |
| -------- | ------- |
| bids_root_path | Full path to root directory of dataset.|
| exclude_dirs | List of directories to exclude when constructing file lists. |
| name_indices  | Indices used by make_file_dict to construct a unique key.<br>(See [Dictionaries of filenames](https://hed-examples.readthedocs.io/en/latest/HedInPython.html#dictionaries-of-filenames-anchor) for examples of how to choose these indices.) |
| skip_columns  | List of columns in the `events.tsv` files to skip in the analysis. |
| value_columns | List of columns in the `events.tsv` files to annotate as<br>as a whole rather than by individual column value. |

**Note:** To actually do the merge of spreadsheet into a JSON sidecar,
you only need to have the file names of the spreadsheet and the JSON sidecar.

For large datasets, you will want to be sure to exclude columns such as
`onset` and `sample`, since the summary produces counts of the number of times
each unique value appears somewhere in an event file.

When run, the script extracts a JSON sidecar and converts it to a spreadsheet
for illustrating the merge. To merge, the spreadsheet is converted back and then
merged with another JSON sidecar.

The example below uses a
[small version](https://github.com/hed-standard/hed-examples/tree/main/datasets/eeg_ds003654s_hed)
of the Wakeman-Hanson face-processing dataset available on openNeuro as
[ds003654](https://openneuro.org/datasets/ds003645/versions/2.0.0).

In [1]:
import os
import json
from hed.tools import EventFileDictionary, EventValueSummary, hed_to_df, df_to_hed, merge_hed_dict
from hed.util import get_file_list

# Variables to set for the specific dataset
bids_root_path =  os.path.realpath('../../../datasets/eeg_ds003654s_hed')
exclude_dirs = ['stimuli']
name_indices = (0, 2)
skip_columns = ["onset", "duration", "sample", "trial", "response_time"]
value_columns = ["rep_lag", "stim_file", "value"]

# Construct the event file dictionary for the BIDS event files
event_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events", exclude_dirs=exclude_dirs)
file_dict = EventFileDictionary(event_files, name_indices=name_indices)

# Get a test spreadsheet for merging
value_sum = EventValueSummary(value_cols=value_columns, skip_cols=skip_columns)
value_sum.update(event_files)
sidecar_template = value_sum.extract_sidecar_template()
str_json = json.dumps(sidecar_template, indent=4)
example_spreadsheet = hed_to_df(sidecar_template)

# Use an empty sidecar to merge into, but any valid sidecar will work
spreadsheet_sidecar = df_to_hed(example_spreadsheet, description_tag=False)
example_sidecar = {}
merge_hed_dict(example_sidecar, spreadsheet_sidecar)
str_json = json.dumps(example_sidecar, indent=4)
print(str_json)

{
    "event_type": {
        "Levels": {
            "double_press": "Description for double_press",
            "left_press": "Description for left_press",
            "right_press": "Description for right_press",
            "setup_right_sym": "Description for setup_right_sym",
            "show_circle": "Description for show_circle",
            "show_cross": "Description for show_cross",
            "show_face": "Description for show_face",
            "show_face_initial": "Description for show_face_initial"
        },
        "HED": {
            "double_press": "Label/double_press",
            "left_press": "Label/left_press",
            "right_press": "Label/right_press",
            "setup_right_sym": "Label/setup_right_sym",
            "show_circle": "Label/show_circle",
            "show_cross": "Label/show_cross",
            "show_face": "Label/show_face",
            "show_face_initial": "Label/show_face_initial"
        }
    },
    "face_type": {
        "Levels": {
